In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pyemu
import flopy
import platform
from pathlib import Path
import shutil
import pandas as pd

In [ ]:
org_ws = Path('../MV_Model_Files/')
new_ws = 'simple_ies'
if "macos" in platform.platform().lower() or "darwin" in platform.platform().lower():
    exe_dir = Path('../bin/mac/')
elif "linux" in platform.platform().lower():
    exe_dir = Path('../bin/linux/')
else:
    exe_dir = Path('../bin/windows/')

In [ ]:
if os.path.exists(new_ws):
    shutil.rmtree(new_ws)
shutil.copytree(org_ws, new_ws)
for f in os.listdir(exe_dir):
    shutil.copy2(os.path.join(exe_dir,f),os.path.join(new_ws,f))

In [ ]:
sim = flopy.mf6.MFSimulation.load(sim_ws = new_ws)

In [ ]:
pyemu.os_utils.run('mf6',cwd=new_ws)

In [ ]:
for f in ["k_aq.ref","k_clay.ref"]:
    arr = np.loadtxt(os.path.join(new_ws,f))
    arr[:,:] = arr.mean()
    np.savetxt(os.path.join(new_ws,f),arr,fmt="%15.6E")

In [ ]:
pyemu.os_utils.run('mf6',cwd=new_ws)

In [ ]:
#need a spatial reference to use pilot points. this means xoff, yoff, rotation, epsg
m=sim.get_model()
m.modelgrid

In [ ]:
pf = pyemu.utils.PstFrom(original_d=new_ws,
                         new_d='template',
                         remove_existing=True,
                         start_datetime="1-1-2025",
                         spatial_reference=m.modelgrid)

In [ ]:
?pf.add_parameters

In [ ]:
k_pars = [f for f in os.listdir(new_ws) if 'k_' in f and f.endswith('.ref')]
k_pars
for f in k_pars:
    pf.add_parameters(f,par_type='constant',upper_bound=20,lower_bound=0.05,pargp="cn-"+f,par_name_base="cn-"+f)

In [ ]:
#for pilot points, we first need to build a variogram, which describes the underlying spatial correlation structure of the variable
pp_v = pyemu.geostats.ExpVario(contribution=1.0, a=5000) #rule of thumb is that range should be atleast 2x pp spacing (delr= 500
pp_gs = pyemu.geostats.GeoStruct(variograms=pp_v,transform='log')
ax=pp_gs.plot()
ax.axvline(x=500*5*5)

In [ ]:
for f in k_pars:
    base = f.split('.')[0]
    pf.add_parameters(f,par_type='pilotpoint',upper_bound=20,lower_bound=0.05,pp_space=3,geostruct=pp_gs,pargp="pp-"+f,par_name_base="pp-"+f)

In [ ]:
?pf.add_observations

In [ ]:
[f for f in os.listdir(new_ws) if f.endswith('.csv')]

In [ ]:
obs_csvs = [f for f in os.listdir(new_ws) if f.endswith('.csv')]
for f in obs_csvs:
    df = pd.read_csv(os.path.join(new_ws,f))
    pf.add_observations(f,index_cols='time',use_cols=df.columns.tolist()[1:])

In [ ]:
for f in k_pars:
    pf.add_observations(f)

In [ ]:
pf.mod_sys_cmds.append('mf6')

In [ ]:
?pf.draw

In [ ]:
pst = pf.build_pst(filename=os.path.join(pf.new_d,'at.pst'),version=2)


In [ ]:
# add in the recharge parameter
r = pst.add_parameters('./simple_ies/at.rch.tpl', pst_path='.')
pars = pst.parameter_data
pars.loc['rch','parval1'] = r.loc['rch'].parval1
pars.loc['rch','parlbnd'] = pars.loc['rch'].parval1 * .8
pars.loc['rch','parubnd'] = pars.loc['rch'].parval1 * 1.2
pars.loc['rch','pargp'] = 'recharge'

pars.loc['rch']

In [ ]:
rb = pst.add_parameters('./simple_ies/riv.ref.tpl', pst_path='.')
rb

In [ ]:
# and the riverbed conductance
pars = pst.parameter_data
pars.loc['rv1','parval1'] = rb.loc['rv1'].parval1
pars.loc['rv1','parlbnd'] = pars.loc['rv1'].parval1 * .1
pars.loc['rv1','parubnd'] = pars.loc['rv1'].parval1 * 10.0
pars.loc['rv1','pargp'] = 'riverbed'

pars.loc['rv1']

In [ ]:
pst.write(os.path.join(pf.new_d,'at.pst'),version=2)

In [ ]:
#draw prior ensemble that accounts for PP spatial correlation (this is for later for when we want to use iES)
pe = pf.draw(num_reals=300)
pe.enforce()
pe.to_csv(os.path.join(pf.new_d,'prior.csv'))



In [ ]:
pst = pyemu.Pst(filename=os.path.join(pf.new_d,'at.pst'))


In [ ]:
pst.pestpp_options["ies_par_en"] = "prior.csv"
obs = pst.observation_data
obs

In [ ]:
pyemu.os_utils.run('pestpp-ies at.pst',cwd='template')

In [ ]:
pst.set_res(os.path.join(pf.new_d,"at.base.rei"))

In [ ]:
pst.phi

In [ ]:
assert pst.phi < 1e-6

In [ ]:
#zero everything out for starters
obs.weight = 0

In [ ]:
#we need to tell PEST what are the observed values and their weights, to calc the objective function
truth = pd.read_csv(os.path.join(new_ws,'obsvalues.dat'),sep=r'\s+')
truth

In [ ]:
obs["standard_deviation"] = np.nan
for i,stuff in truth.iterrows():
    if "headgroup" in stuff.obgnme or "headdiffgroup" in stuff.obgnme or "riv" in stuff.obgnme:
        obs.loc[obs.obsnme.str.contains(stuff.obsnme),'obsval'] = stuff.obsval
        obs.loc[obs.obsnme.str.contains(stuff.obsnme),'weight'] = stuff.weight
        obs.loc[obs.obsnme.str.contains(stuff.obsnme),'obgnme'] = stuff.obgnme
        if "headdiff" in stuff.obgnme:
            obs.loc[obs.obsnme.str.contains(stuff.obsnme),'standard_deviation'] = np.abs(stuff.obsval) * 0.15 
        elif "headgroup" in stuff.obgnme:
            obs.loc[obs.obsnme.str.contains(stuff.obsnme),'standard_deviation'] = 0.5
        else:
            obs.loc[obs.obsnme.str.contains(stuff.obsnme),'standard_deviation'] = np.abs(stuff.obsval) * 0.2 

In [ ]:
obs.loc[obs.weight>0]

In [ ]:
obs.loc[obs.weight>0,:].describe()

In [ ]:
pst.observation_data = obs
pst.write(os.path.join(pf.new_d,'at.pst'),version=2)

In [ ]:
pyemu.os_utils.run('pestpp-ies at.pst',cwd='template')

In [ ]:
phi= pst.phi
phi

In [ ]:
pst.parameter_data.loc[:,"parval1"] = pe.loc[pe.index[0],pst.par_names].values

In [ ]:
pst.parameter_data.head()

In [ ]:
pst.write(os.path.join(pf.new_d,"test.pst"),version=2)
pyemu.os_utils.run("pestpp-ies test.pst",cwd=pf.new_d)

In [ ]:
pst.set_res(os.path.join(pf.new_d,"test.base.rei"))
pst.phi

In [ ]:
kobs = obs.loc[obs.obsnme.str.contains("k_aq"),:].copy()
kobs['i'] = kobs.i.astype(int)
kobs['j'] = kobs.j.astype(int)

In [ ]:
arr = np.zeros((kobs.i.max()+1,kobs.j.max()+1))
arr[kobs.i,kobs.j] = np.log10(pst.res.loc[kobs.obsnme,"modelled"].values)
cb = plt.imshow(arr)
plt.colorbar(cb)

In [ ]:
pst = pyemu.Pst(os.path.join(pf.new_d,"at.pst"))
pst.control_data.noptmax = -2
pst.write(os.path.join(pf.new_d,"at.pst"),version=2)
pyemu.os_utils.run("pestpp-ies at.pst",cwd=pf.new_d)

In [ ]:
pst.set_res(os.path.join(pf.new_d,"at.base.rei"))
pst.phi

In [ ]:
arr = np.zeros((kobs.i.max()+1,kobs.j.max()+1))
arr[kobs.i,kobs.j] = np.log10(pst.res.loc[kobs.obsnme,"modelled"].values)
cb = plt.imshow(arr)
plt.colorbar(cb)